# Homework 1

In [15]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

data_path = "../../datasets"

if torch.accelerator.is_available():
    device = torch.accelerator.current_accelerator()
    print(f"Using accelerator '{device}'")

    if device.type == "cuda":
        torch.backends.cudnn.deterministic = True
else:
    device = torch.device("cpu")
    print("WARN: No accelerator found, running on CPU")


transform = transforms.Compose(
    [
        transforms.ToTensor(),
        # normalize by mean and standard devia,
        transforms.Normalize((0.1307,), (0.3081,)),
    ]
)

train_dataset = datasets.MNIST(
    data_path,
    train=True,
    download=True,
    transform=transform,
)

test_loader = DataLoader(
    datasets.MNIST(data_path, train=False, download=False, transform=transform),
    shuffle=False,
    drop_last=False,
    batch_size=10000,
    generator=torch.Generator(),
)

Using accelerator 'mps'


In [16]:
class MnistCnn(torch.nn.Module):
    def __init__(self):
        super(MnistCnn, self).__init__()

        self.conv1 = torch.nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3)
        self.conv2 = torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.dropout1 = torch.nn.Dropout(p=0.25)
        self.dropout2 = torch.nn.Dropout(p=0.5)
        self.fc1 = torch.nn.Linear(in_features=9216, out_features=128)
        self.fc2 = torch.nn.Linear(in_features=128, out_features=10)

    def forward(self, x):
        x = self.conv1(x)
        x = torch.nn.functional.relu(x)
        x = self.conv2(x)
        x = torch.nn.functional.relu(x)
        x = torch.nn.functional.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = torch.nn.functional.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)

        # Log softmax rather than softmax due to negative log likelihood loss.
        # log_softmax rather than two separate operations for numerical stability
        output = torch.nn.functional.log_softmax(x, dim=1)

        return output

## Part A (12 points)

### Note

When not otherwise specified, use the following parameter values in experiment runs:
- `nr_clients` (N): 100
- `lr`: 0.01
- `client_fraction` (C): 0.1
- `nr_local_epochs` (E): 1
- `batch_size` (B): 100
- `nr_rounds`: 10
- `iid`: True

For all exercises, pass `seed = 10` to calls for splitting data, server initialization, or plotting.

In [17]:
import pandas as pd
import seaborn as sns

clients = 100
learning_rate = 0.01
active_clients_fraction = 0.1
local_epochs = 1
batch_size = 100
rounds = 10
assume_data_iid = True
seed = 10

### Exercise A1: FedSGD with weights (3 points)

#### Question

_(2 points)_ Implement a version of FedSGD that uses weights in its updates, like FedAvg, instead of the gradients from the version of the tutorials. The two FedSGD versions should have the same test accuracy after each round (with a tolerance of up to around 0.1%). To show this, compare their output for the following two scenarios over *5 rounds*:
- `lr = 0.01, client_subsets = split(100, True, ...), client_fraction = 0.5`
- `lr = 0.1, client_subsets = split(50, False, ...), client_fraction = 0.2`

*Tip:* You can use the existing FedAvg implementation to minimize the amount of code writing required.

_(1 point)_ Explain in which cases (for the different decentralized data learning parameters) weight and gradient FedSGD are equivalent.

#### Answer

In [18]:
from components.data_splitting import (
    index_uniformly,
    index_by_approximate_binary_target_partitions,
    partition_dataset,
)
from torch.utils.data import Subset


def determine_client_datasets(iid: bool, clients: int) -> list[Subset]:
    if iid:
        partition_index = index_uniformly(train_dataset, clients, seed)
    else:
        partition_index = index_by_approximate_binary_target_partitions(
            train_dataset, clients, seed
        )

    return partition_dataset(train_dataset, partition_index)

In [19]:
from fedavg import Server as FedAvgServer


def run_fedavg(
    iid: bool, clients: int, active_clients_fraction: float, learning_rate: float
) -> pd.DataFrame:
    partitions = determine_client_datasets(iid, clients)

    server = FedAvgServer(
        device=device,
        model_builder=lambda: MnistCnn().to(device),
        learning_rate=learning_rate,
        batch_size=len(train_dataset),
        client_subsets=partitions,
        client_fraction=active_clients_fraction,
        local_epochs=local_epochs,
        seed=seed,
    )

    return server.run(5, test_loader).as_df()

In [20]:
from fedsgd import Server as FedSgdServer


def run_fedsgd(
    iid: bool, clients: int, active_clients_fraction: float, learning_rate: float
) -> pd.DataFrame:
    partitions = determine_client_datasets(iid, clients)

    fedsgd_server = FedSgdServer(
        device=device,
        model_builder=lambda: MnistCnn().to(device),
        client_subsets=partitions,
        active_clients_fraction=active_clients_fraction,
        learning_rate=learning_rate,
        seed=seed,
    )

    return fedsgd_server.run(5, test_loader).as_df()

In [21]:
fedsgd_results = run_fedsgd(
    iid=True, clients=100, active_clients_fraction=0.1, learning_rate=learning_rate
)

epoch:   0%|          | 0/5 [00:00<?, ?it/s]

clients:   0%|          | 0/10 [00:00<?, ?it/s]

clients:   0%|          | 0/10 [00:00<?, ?it/s]

clients:   0%|          | 0/10 [00:00<?, ?it/s]

clients:   0%|          | 0/10 [00:00<?, ?it/s]

clients:   0%|          | 0/10 [00:00<?, ?it/s]

In [22]:
fedavg_results = run_fedavg(
    iid=True, clients=100, active_clients_fraction=0.1, learning_rate=learning_rate
)

epoch:   0%|          | 0/5 [00:00<?, ?it/s]

clients:   0%|          | 0/10 [00:00<?, ?it/s]

clients:   0%|          | 0/10 [00:00<?, ?it/s]

clients:   0%|          | 0/10 [00:00<?, ?it/s]

clients:   0%|          | 0/10 [00:00<?, ?it/s]

clients:   0%|          | 0/10 [00:00<?, ?it/s]

In [23]:
df = pd.concat([fedsgd_results, fedavg_results])
df.sort_values("round")

,round,algorithm,clients_count,active_clients_fraction,batch_size,local_epochs_count,η,seed,wall_time,message_count (sum),test_accuracy
0,1,FedSgd,100,0.1,∞,1,0.01,10,1.087110,20,0.1247
0,1,FedAvg,100,0.1,60000,1,0.01,10,0.993595,20,0.1247
1,2,FedSgd,100,0.1,∞,1,0.01,10,0.977105,40,0.1519
1,2,FedAvg,100,0.1,60000,1,0.01,10,1.061754,40,0.1519
2,3,FedSgd,100,0.1,∞,1,0.01,10,1.035281,60,0.1816
2,3,FedAvg,100,0.1,60000,1,0.01,10,0.977731,60,0.1816
3,4,FedSgd,100,0.1,∞,1,0.01,10,0.979972,80,0.2176
3,4,FedAvg,100,0.1,60000,1,0.01,10,0.991840,80,0.2176
4,5,FedSgd,100,0.1,∞,1,0.01,10,0.986630,100,0.2573
4,5,FedAvg,100,0.1,60000,1,0.01,10,1.055012,100,0.2573


TODO

### Exercise A2: Client number & fraction (4 points)

#### Question

_(2 points)_ Run the necessary experiments to fill in the following table showing the final message count and test accuracy of FedSGD and FedAvg for different total client numbers:

| Algorithm | N   | C   | Message count | Test accuracy |
| --------- | --- | --- | ------------- | ------------- |
| FedSGD    | 10  | 0.1 |               |               |
| FedAvg    | 10  | 0.1 |               |               |
| FedSGD    | 50  | 0.1 |               |               |
| FedAvg    | 50  | 0.1 |               |               |
| FedSGD    | 100 | 0.1 |               |               |
| FedAvg    | 100 | 0.1 |               |               |

Is the relationship between the metrics/algorithms and client numbers monotonous? Briefly explain your reasoning.

_(2 points)_ Run the experiments to fill in the table when varying the fraction of clients used in every round:

| Algorithm | N   | C    | Message count | Test accuracy |
| --------- | --- | ---- | ------------- | ------------- |
| FedSGD    | 100 | 0.01 |               |               |
| FedAvg    | 100 | 0.01 |               |               |
| FedSGD    | 100 | 0.1  |               |               |
| FedAvg    | 100 | 0.1  |               |               |
| FedSGD    | 100 | 0.2  |               |               |
| FedAvg    | 100 | 0.2  |               |               |

How do the observed relationships compare to before? Again, succintly argue your answer.

#### Answer

In [24]:
# TODO

TODO

In [25]:
# TODO

TODO

### Exercise A3: Local epoch count & (non-)IID data (5 points)

#### Question

_(1 point)_ Create a line plot of the accuracy after each round for the following algorithm variants:

- FedSGD
- FedAvg (E=1)
- FedAvg (E=2)
- FedAvg (E=4)

How does FedAvg compare to FedSGD? What is the effect of increasing the work clients perform locally for each update in FedAvg?

_(2 points)_ Make one line plot of FedSGD and FedAvg under an IID and non-IID split for 15 rounds (leaving all other parameter values as they previously mentioned default). How does the non-IID setting affect the accuracy achieved by the two algorithms? What is the difference in terms of the smoothness of learning?

_(2 points)_ Make another plot for only non-IID splits, including the FedSGD and FedAvg configs from the point above, and add a version for each with a learning rate of 0.001 and client fraction of 0.5. How does the stability of the new variants compare to the ones from before? Why do the changes in parameters have the observed effect?

#### Answer

In [26]:
# TODO

TODO

In [27]:
# TODO

TODO

In [28]:
# TODO

TODO

## Part B (12 points)

### Exercise B1: Microbatch Pipeline Model Parallelism (7 points)

Implement pipeline parallelism with microbatches, as discussed during the lab.

As with the other data/model parallelism examples, you will need a Python script for the nodes and a shell script to orchestrate execution.

Be aware of the possibility of deadlocks: due to how `gloo` operates, it is possible to deadlock by having device 1 send $B_2$ to device 2 in the forward pass, and simultaneously, device 2 send $B_1$ in the backward pass.
Since both sends will await corresponding receives, the training will stop indefinitely.

Use `isend` & `irecv`, the asynchronous (non-blocking) versions of `send` & `recv` in `torch.distributed`.
Each call of the two function returns a `Work` object, on which calling `wait()` blocks, if needed, until the message exchange finishes.
Add comments or text explaining how you expect your implementation to work and test that it runs for the same number of steps and model architecture as in class.

Note that `torch.distributed`'s implementation of `gloo` does not currently support properly asynchronous communication even when using the corresponding primitives.
Thus, you will not see the same improvements in speed as with a backend like `nccl`.

You may also use the fact that `torch` gradients naturally accumulate if zeroed out.
Also, scaling the loss by a constant is equivalent to scaling the resulting gradients by the same constant.

You can rely on receiving messages in the same order they get sent between any device pair.
The `(i)send/recv` functions all support a `tag` attribute to match messages explicitly.
Although using it is good practice, it is not required.

You can refer to the [documentation](https://pytorch.org/docs/stable/distributed.html) and, if helpful, a related [tutorial](https://brsoff.github.io/tutorials/intermediate/dist_tuto.html) on the PyTorch website.

### Exercise B2: Joint Data & Model Parallelism (5 points)

Implement a training setup that uses data and model parallelism together.

Create 2 pipelines of 3 stages running sequentially, where each stage works with 3 sequential micro-batches.

Once again, add comments or text explaining your implementation and test it on the setting that mimics those from the class.

You can use groups from `torch.distributed` to handle operations that require interaction between a subset of more than two but less than all workers.